# Taller 1

Carlos dirige dos sucursales de una empresa nacional de alquiler de carros. Cada dı́a llegan clientes a cada sucursal
con distribuciones $f_1$ y $f_2$ , respectivamente, a alquilar carros. Si al llegar un cliente, Carlos tiene disponible un carro
para alquilar él recibe $r$ por parte de la empresa. Similarmente, cada dı́a llegan clientes a entregar los carros alquilados
en su sucursal o en otras de la misma empresa con distribuciones $g_1$ y $g_2$ respectivamente para cada sucursal. Carlos
solo puede usar los carros devueltos hasta el dı́a siguiente de la entrega pues debe hacerles limpieza. Adicionalmente,
Carlos puede trasladar carros de una sus sucursales a otra a un costo $c$ por carro. Carlos quiere maximizar su
ganancia.

Para modelar este problema con un PDM, elijamos el espacio de estados como $S=\{(s_1,s_2,l_1,l_2)\in \mathbb{N}^4\}$, donde las dos primeras componentes representan el numero de autos que Carlos tiene en cada sucursal y las ultimas dos representan el numero de carros que se terminaron de lavar en el momento de decisión. Para cada uno de estos estados elijamos el conjunto de acciones posibles como $\mathcal{A}_{(s_1,s_2,l_1,l_2)}=\{(i,n):i\in\{1,2\},0\leq n\leq s_i\}$,que representa la acción de sacar del almacen $i$ un número $n$ de autos y llevarlos al otro almacen.

Así, el número de autos disponibles en la sucursal $i$ evolucionaría según la dinámica
\begin{equation}
S_{i,t+1}=[S_{i,t}-Y_{i,t}]^+ +L_{i,t} +u_{i,t},
\end{equation}
que representa el aumento de $S_i$ en $L_{i,t}$ autos lavados, $u_{i,t}$ autos recibidos o enviados a la otra sucursal, y disminuye en $Y_{i,t}$ autos prestados, que son una variable aleatoria de distribución $f_i$. Por otro lado, la cantidad de autos que se están lavando evolucionaría según 
\begin{equation}
L_{i,t+1}=Z_t,
\end{equation}
donde $Z_t$ es una variable aleatoria con distribución $g_i$.

Con esta notación, el sistema de recompensas sería
\begin{equation}
r_t((s_1,s_2,l_1,l_2),(1,n))=r\cdot \mathbb{E}[\min\{Y_{1,t},s_1-n+l_1\}+\min\{Y_{2,t},s_2+n+l_2\}]-n\cdot c
\end{equation}
\begin{equation}
r_t((s_1,s_2,l_1,l_2),(2,n))=r\cdot \mathbb{E}[\min\{Y_{1,t},s_1+n+l_1\}+\min\{Y_{2,t},s_2-n+l_2\}]-n\cdot c
\end{equation}

Y, siguiendo la misma idea del libro, las probabilidades de transición vendrían dadas por
\begin{equation}
p((s_1',s_2',l_1',l_2')|(s_1,s_2,l_1,l_2),(1,n))=g_1(l_1')g_2(l_2')\left(f_1(s_1-s_1'+l_1-n)\sum_{k=0}^{s_1-1}f_1(k))+\delta_{s_1',l_1-n}\sum_{k=s_1}^{\infty}f_1(k)\right)\left(f_2(s_2-s_2'+l_2+n)\sum_{k=0}^{s_2-1}f_2(k))+\delta_{s_2',l_2+n}\sum_{k=s_2}^{\infty}f_2(k)\right)\chi_{n\leq s_1}
\end{equation}

Y 

\begin{equation}
p((s_1',s_2',l_1',l_2')|(s_1,s_2,l_1,l_2),(2,n))=g_1(l_1')g_2(l_2')\left(f_1(s_1-s_1'+l_1+n)\sum_{k=0}^{s_1-1}f_1(k))+\delta_{s_1',l_1+n}\sum_{k=s_1}^{\infty}f_1(k)\right)\left(f_2(s_2-s_2'+l_2-n)\sum_{k=0}^{s_2-1}f_2(k))+\delta_{s_2',l_2-n}\sum_{k=s_2}^{\infty}f_2(k)\right)\chi_{n\leq s_2}
\end{equation}

Bueno, programemoslo

In [79]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from scipy.stats import poisson
from numba import jit

In [136]:
r=10
c=2
f_1=poisson(3)
f_2=poisson(4)
g_1=poisson(3)
g_2=poisson(2)

def dd(i,j):
    if i==j:
        return 1
    else:
        return 0

    
def expect(fa,dis):
    suma=0
    for i in range(10):
        suma+=dis.pmf(i)*fa(i)
    return suma

   
def rec(S,a):
    i=a[0]
    n=a[1]
    s_1=S[0]
    s_2=S[1]
    l_1=S[2]
    l_2=S[3]
    if i==1:
        fa1= lambda y: min(y,s_1-n+l_1)
        fa2= lambda y: min(y,s_2+n+l_2)
        return r*(expect(fa1,f_1)+expect(fa2,f_2))-n*c
    elif i==2:
        fa1= lambda y: min(y,s_1+n+l_1)
        fa2= lambda y: min(y,s_2-n+l_2)
        return r*(expect(fa1,f_1)+expect(fa2,f_2))-n*c

    
def pker(Sp,S,a):
    i=a[0]
    n=a[1]
    s_1=S[0]
    s_2=S[1]
    l_1=S[2]
    l_2=S[3]
    s_1p=Sp[0]
    s_2p=Sp[1]
    l_1p=Sp[2]
    l_2p=Sp[3]
    if i==1:
        if n>s_1:
            
            return 0
        else:
            print(f_1.pmf(s_1-s_1p+l_1-n))
            print(s_1-s_1p+l_1-n)
            return g_1.pmf(l_1p)*g_2.pmf(l_2p)*(
                (f_1.pmf(s_1-s_1p+l_1-n)*f_1.cdf(s_1-1)+dd(s_1p,l_1-n)*f_1.sf(s_1-1)))*(
                (f_2.pmf(s_2-s_2p+l_2+n)*f_2.cdf(s_2-1)+dd(s_2p,l_2+n)*f_2.sf(s_2-1)))
    elif i==2:
        if n>s_2:
            return 0
        else:
            return g_1.pmf(l_1p)*g_2.pmf(l_2p)*(
        (f_1.pmf(s_1-s_1p+l_1+n)*f_1.cdf(s_1-1)+dd(s_1p,l_1+n)*f_1.sf(s_1-1)))*(
        (f_2.pmf(s_2-s_2p+l_2-n)*f_2.cdf(s_2-1)+dd(s_2p,l_2-n)*f_2.sf(s_2-1)))
        

In [140]:
rec([3,4,1,1],[1,1])
pker([3,9,4,4],[7,9,1,1],[1,4])

0.14936120510359185
1


0.00033474042423500513

In [99]:

def L(s,a,S,lamda,v):
    suma=0
    for i in range(len(S)):
        suma+=pker(S[i],s,a)*v[i]
    return rec(s,a)+lamda*suma

     
def value_iteration(S,A,v0,epsilon,lamda):
    v1=v0
    N=0
    while True:
        print(N,np.max(v1-v0))
        popt=[0]*v0.size
        for i in range(len(S)):
            mos=[L(S[i],a,S,lamda,v0) for a in A[i]]
            v1[i]=np.max(mos)
            popt[i]=A[i][np.argmax(mos)]
            print(i)
        if np.max(v1-v0)<epsilon*(1-lamda)/(2*lamda):
            return v1,popt
        v0=v1  
        N+=1

In [100]:
Ss=list(range(4))
Ls=list(range(4))
S=list(itertools.product(Ss, Ss,Ls,Ls))
A=[]
for s in S:
    As=[]
    for s1 in range(s[0]+1):
        As.append([1,s1])
    for s2 in range(s[1]+1):
        As.append([2,s2])
    A.append(As)  
len(S)

256

In [101]:
value_iteration(S,A,np.zeros(len(S)),0.1,0.5)

0 0.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255


(array([ 0.        ,  9.73552118, 18.73841681, 26.27606133,  9.49110444,
        19.22662562, 28.22952125, 35.76716576, 17.48859682, 27.224118  ,
        36.22701363, 43.76465815, 23.24567113, 32.98119231, 41.98408794,
        49.52173245, 18.13763564, 18.82142295, 26.3071498 , 33.76716576,
        19.22662562, 28.22952125, 35.76716576, 41.76465815, 27.224118  ,
        36.22701363, 43.76465815, 49.34863451, 32.98119231, 41.98408794,
        49.52173245, 55.10570882, 26.68996789, 36.37781907, 33.76716576,
        39.76465815, 28.22952125, 35.76716576, 41.76465815, 47.34863451,
        36.22701363, 43.76465815, 49.34863451, 53.10570882, 41.98408794,
        49.52173245, 55.10570882, 58.73601704, 33.78331935, 41.42595643,
        46.56987564, 45.34863451, 35.76716576, 41.76465815, 47.34863451,
        51.10570882, 43.76465815, 49.34863451, 53.10570882, 56.73601704,
        49.52173245, 55.10570882, 58.73601704, 60.80339074, 17.67631658,
        29.56129839, 40.3666499 , 49.27996488, 27.5